# 1. Import Data

In [ ]:
import os 
import pandas as pd
import numpy as np

데이터를 읽고 쓰는데 필요한 라이브러리들을 가져옵니다

________________________
<b>**라이브러리 문서주소 
(필요하시면 보세요. 타이타닉 컴퍼티션 이외에도 다른 데이터 분석을 다룰때 쓰일 수 있는 유용한 function들이 많습니다.)<b>

pandas : <a href=https://pandas.pydata.org/pandas-docs/stable/ target="_blank">판다스 문서</a></p>
numpy : <a href=https://docs.scipy.org/doc/numpy-1.15.0/index.html target="_blank">넘파이 문서</a>
    

In [ ]:
data_dir = '../input'

train_path = os.path.join(data_dir,'train.csv') 
test_path = os.path.join(data_dir,'test.csv') 

raw_train_data = pd.read_csv(train_path)
raw_test_data = pd.read_csv(test_path)


판다스 라이브러리의 read_csv를 이용하여  지정된 경로로 부터csv파일을 읽어 왔습니다.

어떤 데이터가 있는지 어떤 정보를 담고있는지 확인해볼까요?


In [ ]:
raw_train_data.head()
#위에서부터 5개의 raw데이터만 뽑는 것입니다.

In [ ]:
raw_test_data.head()

어떤 데이터들이 있고 어떤식으로 구성 되어있는지 더 자세히 보기 위해 copy함수로 데이터들을 시각화를 위한 변수로 복사하겠습니다.

물론 이번문제에서 쓰일 데이터들은 <b>Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked<b>입니다

In [ ]:
view_train_data = raw_train_data.copy()
view_test_data = raw_test_data.copy()
import matplotlib.pyplot as plt
#시각화를 하기위한 파이썬 라이브러리 matplotlib를 import 합니다.


<b>matplotlib_document: </b><a href=https://matplotlib.org/index.html target="_blank">멧플롯라이브 문서</a>

In [ ]:
view_train_data.describe()
#각 데이터가 가진 통계치입니다. 계산가능한 데이터를 출력하기떄문에 스트링값들은 제외됩니다. 


In [ ]:
view_test_data.describe()

PassengerId에서 이상한 값들이 보이는군요.. Null데이터들이 조금씩 쎃여있는 것같습니다.

# 1.1 Null Check

<b>데이터 분석에 있어서 중요한 것이 null값 데이터입니다. 테이블에 의도되었든 의도되지않았든, ML에서는 이 데이터에 대한 처리없이 그대로 모델에 주게된다면 심각한 문제를 일으킬수있습니다. </b>

In [ ]:
view_train_data.isnull().sum()
#각 에리어에 몇개의 Null 데이터가 있는지 합산해서 보여줍니다.

In [ ]:
for col in view_train_data.columns:
    null_count = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (view_train_data[col].isnull().sum() / view_train_data[col].shape[0]))
    print(null_count)

null값이 있는 것을 확인했습니다.<br>
Cabin의 경우 77프로가 Null값을 가지고있고, 생존률에 영향을 미치는 변수로 보이지않는 좋지 못한 데이터이기 때문에 #Maintenance Part에서 drop 시키겠습니다.<br>
이제 아래의 라이브러리를 사용하여 전체적인 데이터의 시각화를 해보겠습니다.

In [ ]:
import missingno as msno
import seaborn as sns

In [ ]:
msno.matrix(view_train_data, figsize=(7, 7),color=(0.2,0.4,0.8))

In [ ]:
msno.matrix(view_test_data, figsize=(7, 7),color=(0.2,0.8,0.2))

In [ ]:
msno.bar(view_train_data, figsize=(7, 7),color=(0.2,0.4,0.8))

In [ ]:
msno.bar(view_test_data, figsize=(7, 7),color=(0.2,0.8,0.2))

Null 값에 대해 각 피쳐의 중간값으로 채워넣겠습니다.

In [ ]:
#print(train_data.isnull().sum())

data_cleaner = [view_train_data,view_test_data]        

for dataset in data_cleaner:
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0],inplace=True)
    

In [ ]:
print('정리된 후 Null값 확인')
print('Train_data')
print('_______________________________')
print(view_train_data.isnull().sum())
print('\n Test data')
print('_______________________________')
print(view_test_data.isnull().sum())


# 1.2 Label check

<p>train과 test 데이터의 특성을 파악했으니 이제 tabel이라 불리는 탑승자들의 생존 여부, 즉 train data의 답안지를 살펴보도록 하겠습니다.</p>
<li>타이타닉 컴퍼티션의 레이블은 0,1 죽음,생존 의 두가지 binary형식의 label입니다.</li>
<li>labels가 어떤 특성을 가지고 있느냐에 따라 모델의 구성요소가 달라 질수 있습니다.</li>

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

view_train_data['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=view_train_data, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

<p>안타깝게도 38%만 생존하였습니다. 다행히 label의 분포도가 균일한 편입니다.</p>
<p>***1의 label이 99개 0의 lable이 1 같이 label의 분포도가 한쪽으로 쏠린다면 모델의 학습에 큰 지장을 미칠것입니다. 균일한 label이 잡혀있는 것 또한 좋은 데이터셋의 조건중 하나입니다</p>

# 2. Exploratory data analysis

이번에는 label 데이터인 Survivied 피쳐와 각피쳐들이 어떤관계가 있는지 살펴보겠습니다

<h2>Pclass</h2>

<p>먼저 Pclass 에 대해서 살펴보겠습니다. Pclass 는 ordinal, 서수형 데이터입니다. 카테고리이면서, 순서가 있는 데이터 타입입니다.</p>
먼저 Pclass 에 따른 생존률의 차이를 살펴보겠습니다. 엑셀의 피벗 차트와 유사한 작업을 하게 되는데, pandas dataframe 에서는 groupby 를 사용하면 쉽게 할 수 있습니다.

<b>아래와 같이 count() 를 하면, 각 class 에 몇명이 있는 지 확인할 수 있으며, sum() 을 하면, 216 명중 생존한(survived=1)사람의 총합을 주게 됩니다</b>

In [ ]:
view_train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count()

In [ ]:
view_train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).sum()

In [ ]:
pd.crosstab(view_train_data['Pclass'], view_train_data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

pandas의 crosstab 을 사용하면 좀더 세부적으로 표현할수 있습니다.

In [ ]:
view_train_data[['Pclass','Survived']].groupby(['Pclass'], as_index=True).mean().plot.bar()
#각 pclass의 테이블의 속성에 대한 평균을 구한후 그래프로 구현함(ex.1의 경우 136/216으로 계산)

In [ ]:
sns.countplot('Pclass', hue='Survived', data=view_train_data)

pclass의 등급이 높을수록 생존률이 높은것을 확인 할수 있습니다

<h2>Sex</h2>

이번에는 성별(sex)와 생존률의 관계에 대해 알아보겠습니다. 성별은 순서가없는 범주형 데이터로 남/녀 구분만 하면 되므로 앞의 pclass와는 다르게 비교적 쉽게 관계를 알아볼수 있습니다.

In [ ]:
view_train_data[['Sex','Survived']].groupby(['Sex']).mean()

In [ ]:
f,ax = plt.subplots(1,2,figsize=(17,7))
view_train_data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
sns.countplot('Sex', hue='Survived', data=view_train_data,ax=ax[1])

남자에 비해 여자의 생존률이 높은 것을 확인할 수 있습니다.

<h2>Age</h2>

<li>나이와 생존률에 대한 관계를 알아보겠습니다.</li>
<p>우선 </p>

In [ ]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(view_train_data['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(view_train_data['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(view_train_data['Age'].mean()))

아래는 나이에 따른 생존률에 관한 histogram입니다.

In [ ]:
#view_train_data[view_train_data['Survived']==1]['Age']
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(view_train_data[view_train_data['Survived']==0]['Age'],ax = ax)
sns.kdeplot(view_train_data[view_train_data['Survived']==1]['Age'],ax=ax)
plt.legend(['Survived==0','Survived==1'])
plt.show()

저 연령층(20세 미만, 나이값이 낮을 수록)이 생존률이 높은 것을 확인할 수가 있습니다.

<h3>Age distribution with Pclass</h3>

각 Pclass 피쳐에 있는 나이대가 어떤지 한번 살펴보죠

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(view_train_data[view_train_data['Pclass']==1]['Age'])
sns.kdeplot(view_train_data[view_train_data['Pclass']==2]['Age'])
sns.kdeplot(view_train_data[view_train_data['Pclass']==3]['Age'])
plt.title('Age distribution with plcass')
plt.legend(['pclass==1','pclass==2','pclass==3'])
plt.show()

Pclass가 높을 수록 고 연령대가 많이 분포되어있는 것이 보이는 군요. 
저 연령층은 대부분 2,3class에 있네요

<h3>Sex distribution with Pclass,survived and Age</h3>

방금까지 리뷰했던 성별 ,나이, pclass 데이터들의 관계를 이용하여  생존률이 얼마나 되는지 알아보겠습니다. 

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,7))
sns.violinplot('Pclass','Age',hue='Survived',data=view_train_data,scale='count',split=True,ax=ax[0])
ax[0].set_title('Pclass and Age relation with Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot('Sex','Age',hue='Survived',data=view_train_data,scale='count',split=True,ax=ax[1])
ax[1].set_yticks(range(0,110,10))
ax[1].set_title('Sex and Age relation with Survived')
plt.show()

pclass가 높을수록,나이가 어릴 수록 그리고 남성 보다는 여성이 더 많은 생존률을 보여주고있습니다. <br>
***이것은 침몰 당시 여성과 아이를 먼저 챙긴것을 나타내고 있습니다.

<h2>Embarked</h2>

Embarked는 타이타닉호를 탑승한 항구를 나타냅니다.<br>
위에서 한것 처럼 탑승한 곳에 따른 생존률을 표시해보겠습니다

In [ ]:
view_train_data[['Embarked','Survived']].groupby(['Embarked']).mean()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
view_train_data[['Embarked','Survived']].groupby(['Embarked']).mean().plot.bar(ax=ax)
ax.set_title('Relationship Embarked  and Survived')

C가 다른 항구에 비해서 약간 생존률이 더 높은 것을 확인할 수가 있습니다.<br>
전체적으로 비슷한 데이터이고 탑승한 항구가 생존률에 어떤 영향을 미칠수 있는 지는 모르겠지만,<br>
데이터 분포가 나쁘지않기때문에 사용하겠습니다. 조금 더 관계를 알아보도록 하죠

<h3>Relationship Embarked with other data</h3>

탑승한 항구에 대해 성별,Pclass,생존률 에 대한 관계를 각각의 데이터로 시각화하여 알아보겠습니다.<br>
우선 탑승한 항구에 대한 각데이터들의 분포부터 살펴보겠습니다.

In [ ]:
fig,ax =plt.subplots(2,2, figsize=(20,10))
sns.countplot('Embarked',data=view_train_data,ax=ax[0][0])
ax[0][0].set_title('The number of Passengers who take a ship by embarked')
sns.countplot('Embarked',hue='Survived',data=view_train_data,ax=ax[0][1])
ax[0][1].set_title('The number of Survived Passengers ')
sns.countplot('Embarked',hue='Pclass',data=view_train_data,ax=ax[1][0])
ax[1][0].set_title('The number of  Passengers  relation with embarked which is classified by pclass ')
sns.countplot('Embarked',hue='Sex',data=view_train_data,ax=ax[1][1])
ax[1][1].set_title('The number of  Passengers  relation with embarked which is classified by Sex ')

<li>The number of Passengers who take a ship by embarked : S항구에서 가장많은 탑승자가 탑승했습니다.</li>
<li>The number of Survived Passengers : S항구에서 가장많은 탑승자의 사망배율이 매우 높은편이며 각 항구 클래스의 생존률만 보았을때는 C항구에서 탑승한 사람들의 값이 가장 높습니다.</li>
 <li>The number of  Passengers  relation with embarked which is classified by pclass : 역시 S항구가 가장많은 탑승자를 보유하고있고 3class의 좌석에 탑승한 사람들이 가장 많습니다. 반면 c항구에서는 1번 클래스의 좌석에 탑승한 사람들이 많습니다.(이로써 pclass등급에 따른 생존률은 비례한다는 가설은 맞는 것 같군요..)</li> 
 <li>The number of  Passengers  relation with embarked which is classified by Sex : 전체적으로 남성이 여성보다 높은 탑승률을 보이며 특히나 S항구에서 남성 탑승자가 여성보다 더 많은것을 알수 있습니다.</li>

# 2.1Maintenance for Data



 <li>데이터 분석에서 좀 더 명확한 데이터 정제를 위해 기본 데이터에 약간의 변형을 줄 수있습니다.(변형되는 데이터가 가르키는 정보는 변형된 후의 데이터의 정보가 가르키는것과 같아야합니다).</li>
 <li>새로운 피쳐가 만들어질 수도있고 쓸모없는 피쳐들은 drop시키는 등의 변형입니다. </li>
<li>다만, 제공되는 데이터의 직접적인 변형이 아닌, 프로그램 내에 저장된 변수의 변형이 캐글에서는 좋은 접근방법입니다.(항상 이런건 아닙니다.)</li>
 **우선 데이터의 상태를 보고 시작하겠습니다.

In [ ]:
view_train_data.isnull().sum()

In [ ]:
view_test_data.isnull().sum()

test랑 train 테이블에서 Cabin 피쳐가 유난히 많은 Null값을 가지고있는 것이 보입니다. 그러면 쓸모없어보이는 피쳐인 PassengerId와 Cabin,Ticket을 드롭시키겠습니다.<br>
**초반에 시각화를 위해 사용하였던 data_clear 리스트를 사용하겠습니다.(사실 저것도 maintendance의 작업중 하나입니다만, 시각화를 위해 어쩔수 없이 사용했습니다.

In [ ]:
drop_column = ['PassengerId','Cabin', 'Ticket']
view_test_data.drop(drop_column, axis=1, inplace = True)
view_train_data.drop(drop_column, axis=1, inplace = True)


In [ ]:
view_test_data.info()
print('________________________________________________________')
print("PassengerId,Cabin, Ticket의 피쳐가 각 테이블 셋으로부터 제거되었습니다.")

<h3>SibSp, Parch,Name and new feature Familysize</h3>

<p>SibSp은 함께 탑승한 형제,자매, Parch은 부모,자녀의 수를 나타내는 피쳐입니다. 다만 본인은 제외 되어있고 혼자 탑승할시에 둘다. 0,0이 됩니다.</p>
<p>함께 탑승한 가족수(자신 포함)을 명확하게 표현하기 위해 Familysize=Parch은+Sibsp+1 이라는 법칙을 적용한 새로운 피쳐를 생성하겠습니다. </p>
<p>혼자인 사람들을 구분하기 위해 alone 피쳐를 만들겠습니다.( 0,이면 여러명,1이면 혼자) </p>
<p>한국과 다른 서양에서만 있는 이름에 관한 특징 사용하자면 이름앞에 칭호가 붙습니다(mrs:결혼한 여성,master:미성년 아들, dr:박사 등등 </p>

In [ ]:
for dataset in data_cleaner:
    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1 
    
    dataset['Alone'] = 1
    
    dataset['Alone'].loc[dataset['FamilySize'] > 1] = 0
        
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".",expand=True)[0]
    

<li>FamilySize의 경우 그대로 Parch은+Sibsp+1의 값을 넣었습니다.</li>
<li>Alone의 경우 우선 모두다 1값으로 채워넣고 각 raw마다 (특정조건)familysize의 값이 1보다 클경우 0으로 부분적인 마스킹을 합니다</li>
<li>Title이라는 새로운 피쳐를 생성하고 Name피쳐에서 칭호(미스터 미세스 등등)만 분리하여 대입 시킵니다.</li>

In [ ]:
title_names = (view_train_data['Title'].value_counts() < 10)

view_train_data['Title'] = view_train_data['Title'].apply(lambda x:'Misc' if title_names.loc[x] == True else x)


In [ ]:
view_train_data[['Title','Alone','FamilySize']].head()

mr,miss,mrs밖에 안나왔지만 혹시나 모르니 이름쪽을 한번 더 보겠습니다.

In [ ]:
pd.crosstab(view_train_data['Title'], view_train_data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

<p>captin, sir,rev 등등 생각보다 많은 호칭들이 있습니다. 이러한 호칭들은 사실 name 피쳐로부터 파생된 나온것이고, 저희의 의도가 결혼을 했냐 아니냐 등 가족력을 분별하기 위한 것이거나 생존률에 영향을 미칠만큼 많이 모여있지않는(노이즈를 발생시킬수 있는) 항목들은 정리를 하겠습니다. </p>
<p>***10보다 갯수가 적은 타이틀들은 전부 노이즈로 취급하고 Misc라는 새로운 타이틀로 통합하여 데이터에 넘겨주겠습니다.</p>

In [ ]:
title_names = (view_train_data['Title'].value_counts() < 10)

view_train_data['Title'] = view_train_data['Title'].apply(lambda x:'Misc' if title_names.loc[x] == True else x)


In [ ]:
pd.crosstab(view_train_data['Title'], view_train_data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

이제 Name피쳐의 가치는 없으니 드롭시키도록하겠습니다.

In [ ]:
view_train_data.drop('Name', axis=1, inplace = True)
view_test_data.drop('Name', axis=1, inplace = True)

In [ ]:
view_test_data.head()

  <h3>Fare</h3>

마지막으로 요금 데이터를 살펴보겠습니다.

In [ ]:
sns.distplot(view_train_data['Fare'])

<li>데이터가 한쪽으로 너무 쏠려있고 fare의 범위값이 0~500으로 매우 광범위한 것을 알수 있습니다. 이러한 데이터는 이후 모델을 학습할때 모델의 성능을 크게 저하시킬수 있습니다.</li>
<br>**데이터 정리가 필요한 시점입니다. 튀지않도록 평균으로 뺀 절대값을 구한후 표준편차로 나누겠습니다. 이러한과정을 정규화라고 합니다
<p>요금 말고도 다른 값들도 튀는 값들이 있으니 전체 데이터에 대한 정규화를 다음절에서 하도록 하겠습니다</p>

In [ ]:
def data_regularization(data):
    cols = list(data.columns)
    for col in cols:
        std = data[col].std()
        mean = data[col].mean()
        data[col] = np.abs(data[col] - mean)
        data[col] /= std
    return data



<h3>Maintenance</h3>

<li>정규화를 진행하기전, 우선 특정 피쳐들에 대해 계산이 가능하도록 숫자로 바꿔보겠습니다.</li>
<li>Training data에 있는 Survived 피쳐를 이제 train_label로서 따로 떼어내겠습니다.</li>
<li>sklearn의 LabelEncoder 객체를 사용하겠습니다.</li>


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for dataset in data_cleaner:
    dataset['Title'] =  label_encoder.fit_transform(dataset['Title'])
    dataset['Sex'] =  label_encoder.fit_transform(dataset['Sex'])
    dataset['Embarked'] =  label_encoder.fit_transform(dataset['Embarked'])
    dataset['Embarked'] =  label_encoder.fit_transform(dataset['Embarked'])
train_label =np.array(view_train_data.pop('Survived').tolist())

이제 데이터 정규화를 합니다

In [ ]:
view_train_data = data_regularization(view_train_data)
view_test_data = data_regularization(view_test_data)

In [ ]:
view_train_data.info()

In [ ]:
view_test_data.info()

In [ ]:
view_train_data.head()

In [ ]:
sns.distplot(view_train_data['Fare'])

0부터 500까지 였던 데이터의범위가 0~6으로 크게줄은 것을 볼수있습니다. 이제 본격적으로 모델을 정의한 후 계산을 시작해보겠습니다.

In [ ]:
from keras import *
from keras.models import *

from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

여기서는 가장  basic한 딥러닝의 fully-connected layers로 풀것입니다.(keras library 사용)<br>
다른 svc,AdaBoost 등 유명한 ML -classifer들을 import 하는 이유는 FC모델과의 성능 비교를 위함입니다.

In [ ]:
X_tr, X_vld, y_tr, y_vld = train_test_split(view_train_data, train_label, test_size=0.25, random_state=2019)

간단한 문법설명을 드리자면, sklearn에서 df로부터 train과 validation을 분리하는 역할을 하게하는 함수입니다. random_state는 데이터를 무작위로 쎃는 역할인데 2018회만큼 셔플하고 test_size는 전체 train중에 20프로를 떼어내어 훈련이 끝날때마다 어떻게 학습되어가는가 검증하는 데이터로 쓰입니다.(고등 학생 때 수능을 치기 위해 공부했던 우리들의 실력검증을 위해 주기마다 치뤄지는 모의고사랑 같은 개념입니다.)<br>

In [ ]:
##models
def Vanila_fully_connected_layer_model():
    model = Sequential()
    model.add(layers.Dense(256,activation='relu',input_shape=(10,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512,activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(0.7))
    model.add(layers.Dense(2048,activation='relu'))
    model.add(layers.Dropout(0.7))
    model.add(layers.Dense(2048,activation='relu'))
    model.add(layers.Dropout(0.7))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    history= model.fit(x=X_tr,y=y_tr,epochs=20,validation_data=(X_vld,y_vld))
    return model,history


def Adaboost_classifier():
    model=AdaBoostClassifier()
    model.fit(X_tr, y_tr)
    prediction = model.predict(X_vld)
    return model.score(X_vld, y_vld)

def svc_model():
    model =LinearSVC()
    model.fit(X_tr, y_tr)
    prediction = model.predict(X_vld)
    return model.score(X_vld, y_vld)

def random_forest_classfier():
    model = RandomForestClassifier()
    model.fit(X_tr, y_tr)
    prediction = model.predict(X_vld)
    return model.score(X_vld, y_vld)



모델들을 함수로써 정의합니다.

In [ ]:
vanila_model,vanila_history = Vanila_fully_connected_layer_model()

In [ ]:
acc = vanila_history.history['acc']
val_acc = vanila_history.history['val_acc']
loss= vanila_history.history['loss']
val_loss = vanila_history.history['val_loss']

epochs = range(1,len(acc)+1)

In [ ]:
plt.plot(epochs,acc,"bo")
plt.plot(epochs,val_acc,"b")
plt.legend(["train_accuracy","validation_accuracy"])
plt.show()

In [ ]:
plt.plot(epochs,loss,"bo")
plt.plot(epochs,val_loss,"b")
plt.legend(["train_loss","validation_loss"])
plt.show()

80프로대가 조금 아쉽지만, dropout만을 첨가한 fully-connected layer 치고는 괜찮은 성능이 나오는거 같습니다. 그럼! 다른 유명한 classifer들과 비교해보죠!

In [ ]:
others_models_results = [svc_model(),Adaboost_classifier(),random_forest_classfier(),vanila_model.evaluate(X_vld,y_vld)[1]]
plt.plot(range(1,len(others_models_results)+1),others_models_results,"bo")
plt.show()
print("순서대로 svc,Adaboost,random_forest,Fc_layer 입니다.")

In [ ]:
print(others_models_results)
print("84프로면 나쁘지않은 결과이군요 ")

# 3.결론

지금까지 저희가 문제를 풀면서 가졌던 궁극 적인 목표입니다. 이러한 과정들 속에서 결국은 어떤 피쳐가 생존에 가장 큰 영향을 주었나를 파악하는 것입니다.<br>
제가 사용했던 keras library가 그런 feature importance 쪽은 지원되는 라이브러리가 없기때문에 sklearn의 randomforest모델을 새로 정의하여 이것을 통해 결론짓겠습니다<br>
***pandas 라이브러리의 Series 객체는 각 개체의 특성별로 차트를 출력하는 여러가지 function들을 담고있는 객체입니다.
    

In [ ]:
from pandas import Series

skmodel = RandomForestClassifier()
skmodel.fit(X_tr, y_tr)

feature_importance = skmodel.feature_importances_
Series_feat_imp = Series(feature_importance, index=view_test_data.columns)

In [ ]:
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

요금(돈이 가장많은),나이, 성별(여자 쪽),Title( 칭호- 미혼인가 기혼인가), pclass(좌석등급 -역시 돈관련) 순으로 생존률에 영향을 미침을 보여줍니다.<br>
저 그래프는 randomforest에 모델이 저 변수들을 우선적으로 따진다는 것이 됩니다.<br>
자 이제 결론을 짓고 답안지를 제출하러 갈 차례군요.<br>
<li>이제까지 언급하지않았던 submission.csv 파일은 답안제출용 파일입니다.</li>
<li>이 파일을 import 하여 여기에 test_data에 대한 결과값을 넣고 캐글 커널을 올리면 대회 답안 제출이 되어지는 것입니다!</li>

In [ ]:
prediction = vanila_model.predict(view_test_data)
submission = pd.read_csv(os.path.join(data_dir,'sample_submission.csv'))
submission['Survived'] =prediction.round()

submission.to_csv("./submission.csv", header=True, index=False)

자 이상태로 캐글 커널로 올리면 대회문제를 푼것은  submission 파일을 답지로 올리면 됩니다.<br>

부족한 강의 들어주셔서 감사합니다 수고하셨습니다.^^

